In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision
from torchvision import models, transforms

In [2]:
# 학습된 VGG-16 모델 읽어오기
net = models.vgg16(pretrained = True)
net.eval() # 추론(평가) 모드로  설정 - 드랍아웃 및 배치 정규화를 평가 모드로 설정...

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
# 데이터 전처리
# PIL  (높이,너비,채널)  파이토치(채널,높이,너비)
class BaseTransform():
  def __init__(self,resize,mean,std):
    self.base_transform = transforms.Compose([
        transforms.Resize(resize), # 짧은 변의 길이가 resize가 되도록 만든다.
        transforms.CenterCrop(resize),  # 화면 중앙을 resize x resize로 자른다
        transforms.ToTensor(),
        transforms.Normalize(mean,std)  # 색상정보를 표준화
    ])
  def __call__(self,img):    
    return self.base_transform(img)

In [8]:
# vgg-16 1000차원 출력을 라벨명으로 변환 ILSVRPredictor 클래스를
# json 파일
import json
ILSVRC_class_index =  json.load(open('./imagenet_class_index.json','r'))

In [9]:
# 출력 결과에서 라벨을 예측하는 클래스
class ILSVRCPredictor():
  def __init__(self,class_index):
    self.class_index = class_index
  def predict_max(self, out):
    # 최대확률의 라벨명을 가져온다
    maxid = np.argmax(out.detach().numpy())  # 출력 값을 네트웍에서 분리
    predicted_label_name = self.class_index[str(maxid)][1]
    return predicted_label_name

In [ ]:
# 이미지 읽어오기
resize=224
mean,std = (0.5,0.5,0.5),(0.2,0.2,0.2)
img_path = '/content/다운로드.jpg'
img = Image.open(img_path)
transform = BaseTransform(resize,mean,std)
img_transformed = transform(img)
print(img_transformed.shape)
inputs = img_transformed.unsqueeze(0)
inputs.shape

In [ ]:
out = net(inputs)
predictor = ILSVRCPredictor(ILSVRC_class_index)
result = predictor.predict_max(out)
result